In [ ]:
# ==============================================================================
# CELL 1: Install all required libraries
# ==============================================================================
!pip install gradio python-dotenv julep pyyaml -U --quiet
print("Libraries installed successfully!")

# ==============================================================================
# CELL 2: Import libraries and handle API keys securely
# ==============================================================================
import os
import uuid
import time
import yaml
import gradio as gr
from julep import Julep
import getpass # Used for securely inputting API keys in Colab

# --- Secure API key input ---
print("🔑 Please enter your API keys.")
JULEP_API_KEY = getpass.getpass("Enter your Julep API Key: ")
OPENWEATHERMAP_API_KEY = getpass.getpass("Enter your OpenWeatherMap API Key: ")

# Initialize Julep client
client = Julep(api_key=JULEP_API_KEY)
print("\n Julep client initialized.")


# ==============================================================================
# CELL 3: Define the Julep Agent and Gradio Interface
# ==============================================================================

# YAML template for the Julep agent task
task_yaml_template = """
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Personalized Dining Advisor
description: Suggests dining options based on weather, cuisine, budget, and dietary needs.

input_schema:
  type: object
  properties:
    locations:
      type: array
      items:
        type: string
    date:
      type: string
    budget:
      type: string
    dietary_preference:
      type: string

tools:
- name: weather_tool
  type: integration
  integration:
    provider: weather
    setup:
      openweathermap_api_key: "{openweather_api_key}"

- name: wiki_tool
  type: integration
  integration:
    provider: wikipedia

main:
- over: $ steps[0].input.locations
  map:
    tool: weather_tool
    arguments:
      location: $ _
      date: $ steps[0].input.date

- over: $ steps[0].input.locations
  map:
    tool: wiki_tool
    arguments:
      query: $ 'cuisine of ' + _

- evaluate:
    results: |-
      $ list(
        zip(
          steps[0].input.locations,
          steps[0].output,
          steps[1].output
        )
      )

- over: $ _['results']
  map:
    prompt:
    - role: system
      content: >-
        You are an expert culinary travel assistant. For each city, using the provided info, you MUST follow these rules:
        1.  **Dining Style:** Suggest dining **indoors** or **outdoors** based on weather (Indoor for rain/snow/wind>8m/s/temp<10°C or >35°C; Outdoor for clear/calm/temp 15–30°C).
        2.  **Dishes:** Extract 3 top traditional dishes that match the user's dietary preference of **{{steps[0].input.dietary_preference}}**. If the preference is 'Any', provide a mix. If 'Veg', only provide vegetarian dishes. If 'Non-Veg', prioritize meat/fish dishes.
        3.  **Restaurants:** Recommend 2 restaurants for each dish. The restaurants MUST match the user's budget of **{{steps[0].input.budget}}** (e.g., Low, Moderate, or High) and the appropriate indoor/outdoor seating.
        4.  **Food Tour:** Create a "One-Day Food Tour Itinerary" (Breakfast, Lunch, Dinner) using the dishes and restaurants you just identified.

        Present all information in clean, readable markdown format. Start each city with a main heading.

    - role: user
      content: >-
          City: {{_[0]}}
          Weather: {{_[1]}}
          Cuisine Info: {{_[2]}}
          Budget: {{steps[0].input.budget}}
          Dietary Preference: {{steps[0].input.dietary_preference}}

    unwrap: true

- evaluate:
    final_report: |-
      $ "\\n---\\n".join(_)
"""

# --- MODIFIED FUNCTION WITH ENHANCED PROGRESS TRACKING ---
def get_dining_advice(locations, date, budget, dietary_preference, progress=gr.Progress(track_tqdm=True)):
    """
    This function takes user input from Gradio, runs the Julep agent task,
    and returns the final report, showing progress along the way.
    """
    if not locations or not date:
        return "Please select at least one location and enter a date."

    # The 'budget' input from the gr.Radio component is already a string ("Low", "Moderate", or "High").
    # No mapping is needed.

    progress(0, desc=" Initializing Agent...")

    agent_id = str(uuid.uuid4())
    task_id = str(uuid.uuid4())

    try:
        progress(0.1, desc=" Configuring Agent...")
        client.agents.create_or_update(
            agent_id=agent_id,
            name="Personalized Dining Advisor",
            about="Suggests dining options based on weather, cuisine, budget, and dietary needs.",
            model="gpt-4o"
        )

        progress(0.2, desc="📝 Preparing Task...")
        task_yaml_string = task_yaml_template.format(openweather_api_key=OPENWEATHERMAP_API_KEY)
        task_yaml = yaml.safe_load(task_yaml_string)
        client.tasks.create_or_update(
            task_id=task_id,
            agent_id=agent_id,
            **task_yaml
        )

        progress(0.3, desc=f"📡 Fetching data for {', '.join(locations)}...")
        execution = client.executions.create(
            task_id=task_id,
            input={
                "locations": locations,
                "date": date,
                "budget": budget, # Use the budget string directly
                "dietary_preference": dietary_preference
            }
        )

        # --- Enhanced Progress Loop ---
        current_progress = 0.3
        while (result := client.executions.get(execution.id)).status not in ['succeeded', 'failed']:
            # Increment progress slightly to show activity, capping at 0.9
            current_progress = min(current_progress + 0.05, 0.9)
            progress(current_progress, desc=f" Agent is working... (Status: {result.status})")
            time.sleep(2)

        progress(1, desc=" Done!")
        if result.status == "succeeded":
            return result.output.get('final_report', 'No final report available.')
        else:
            error_message = f" **Error:** {result.error}\n"
            if hasattr(result, 'logs') and result.logs:
                log_str = str(result.logs).replace('\\n', '\n')
                error_message += f"🪵 **Logs:**\n```\n{log_str}\n```"
            return error_message

    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

# --- Create the Gradio Interface (Your UI code is already great!) ---

with gr.Blocks(theme=gr.themes.Soft(), title="Dining Advisor Agent") as demo:
    gr.Markdown(
        """
        # 🍽️ Personalized Dining Advisor
        Select cities, a date, your budget, and dietary preferences to get a custom-tailored dining plan from our Julep AI Agent.
        """
    )

    with gr.Row():
        locations_input = gr.CheckboxGroup(
            choices=["Pune", "Mumbai", "New York", "London", "Tokyo", "Paris", "Rome", "San Francisco"],
            label="Select Cities"
        )
        with gr.Column():
            date_input = gr.Textbox(label="Date", placeholder="e.g., 2025-07-05")
            budget_input = gr.Radio(
                choices=["Low", "Moderate", "High"],
                label="Budget",
                value="Moderate"
            )
            dietary_input = gr.Radio(
                choices=["Any", "Veg", "Non-Veg"],
                label="Dietary Preference",
                value="Any"
            )

    submit_button = gr.Button("Get Personalized Advice", variant="primary")
    output_report = gr.Markdown(label="Your Personalized Plan")

    submit_button.click(
        fn=get_dining_advice,
        inputs=[locations_input, date_input, budget_input, dietary_input],
        outputs=output_report
    )

    gr.Examples(
        examples=[
            [["Pune", "Mumbai"], "2025-07-08", "Moderate", "Any"],
        ],
        inputs=[locations_input, date_input, budget_input, dietary_input]
    )

# ==============================================================================
# CELL 4: Launch the Gradio App
# ==============================================================================
print("\n🚀 Launching Gradio App... Click the public URL to open the interface.")
demo.launch(debug=True, share=True)

Libraries installed successfully!
🔑 Please enter your API keys.
Enter your Julep API Key: ··········
Enter your OpenWeatherMap API Key: ··········

 Julep client initialized.

🚀 Launching Gradio App... Click the public URL to open the interface.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://77c14f37b5a9691323.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
